# Investment Simulation System

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
%pip install pandas_ta_classic

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tqdm 

Note: you may need to restart the kernel to use updated packages.


### Imports

In [4]:
import os
import site

import os
import shutil
import gc
import warnings
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm
from datetime import timedelta

warnings.filterwarnings('ignore')

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

from sklearn.metrics import (
    precision_score, recall_score, f1_score, matthews_corrcoef,
    mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
)


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [7]:
pandas_ta_classic_path = None
for sp in site.getsitepackages():
    pandas_ta_classic_path = os.path.join(sp, 'pandas_ta_classic')
    if os.path.exists(pandas_ta_classic_path):
        break

if pandas_ta_classic_path:
    squeeze_pro_path = os.path.join(pandas_ta_classic_path, 'momentum', 'squeeze_pro.py')
    if os.path.exists(squeeze_pro_path):
        try:
            with open(squeeze_pro_path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            fixed = False
            for line in lines:
                if "from numpy import NaN as npNaN" in line:
                    new_lines.append(line.replace("from numpy import NaN as npNaN", "# from numpy import NaN as npNaN\nimport numpy as np\n"))
                    fixed = True
                    print("Modified import statement in squeeze_pro.py")
                else:
                    new_lines.append(line)

            if fixed:
                with open(squeeze_pro_path, 'w') as f:
                    f.writelines(new_lines)
                print("Successfully patched pandas_ta_classic/momentum/squeeze_pro.py")
            else:
                print("Could not find the problematic import line in squeeze_pro.py")

        except Exception as e:
            print(f"Error modifying squeeze_pro.py: {e}")
    else:
        print(f"Could not find squeeze_pro.py at {squeeze_pro_path}")
else:
    print("Could not find the pandas_ta_classic library installation path.")

import pandas_ta_classic as ta

Could not find the problematic import line in squeeze_pro.py


### Configurations

In [8]:
MODEL_SAVE_PATH = "trained_models/"
MIN_SEQUENCE_LENGTH = 12  # Minimum sequence length for any company
MAX_SEQUENCE_LENGTH = 12  # Maximum sequence length to cap computational cost
INITIAL_TRAINING_DAYS = 1100  # Number of days to use for initial training only
KELLY_FRACTION = 0.05
SECTOR_CONFIDENCE_THRESHOLD = 0.40
RETRAIN_INTERVAL = 200
MAX_DAY_GAP = 5  # Maximum allowed gap in trading days (to account for weekends/holidays)

#### Identify Contiguous Periods
Groups rows for a ticker into blocks where data gaps never exceed MAX_DAY_GAP

In [9]:
def identify_contiguous_periods(df: pd.DataFrame, max_gap_days: int = MAX_DAY_GAP) -> list:

    if df.empty:
        return []

    df = df.sort_values('date').reset_index(drop=True)
    dates = pd.to_datetime(df['date'])

    contiguous_periods = []
    start_idx = 0

    for i in range(1, len(dates)):
        gap = (dates[i] - dates[i-1]).days
        if gap > max_gap_days:
            # End current period and start new one
            contiguous_periods.append((start_idx, i-1))
            start_idx = i


    contiguous_periods.append((start_idx, len(dates)-1))


    contiguous_periods = [(s, e) for s, e in contiguous_periods if e - s >= MIN_SEQUENCE_LENGTH]

    return contiguous_periods

#### Calculate Dynamic Sequence Length 
Pick a sequence length tailored to the amount of history available for a ticker.
Avoids hard-coding a single window size for short vs long histories.

In [10]:
def calculate_dynamic_sequence_length(company_data_df: pd.DataFrame,
                                     min_length: int = MIN_SEQUENCE_LENGTH,
                                     max_length: int = MAX_SEQUENCE_LENGTH,
                                     target_fraction: float = 0.15) -> int:

    total_days = len(company_data_df)


    dynamic_length = int(total_days * target_fraction)
    dynamic_length = max(min_length, min(dynamic_length, max_length))

    return dynamic_length

#### Create Contiguous Sequences
Turn each contiguous period into overlapping (sequence_length) windows (X) and next-step labels (y).
Produces the actual training/validation tensors for the model.

In [11]:
def create_contiguous_sequences(data: np.ndarray, targets: np.ndarray,
                               contiguous_periods: list, sequence_length: int):

    X, y = [], []

    for start_idx, end_idx in contiguous_periods:
        period_length = end_idx - start_idx + 1
        if period_length < sequence_length:
            continue

        for i in range(start_idx + sequence_length, end_idx + 1):
            X.append(data[i-sequence_length:i])
            y.append(targets[i])

    return np.array(X) if X else np.array([]), np.array(y) if y else np.array([])

#### Create Target Variable
Build the binary classification target per row.

In [12]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:

    print("Creating target variable...")
    df = df.sort_values(by=['ticker', 'date']).copy()
    df['next_day_close'] = df.groupby('ticker')['close_price'].shift(-1)
    df['target'] = (df['next_day_close'] > df['close_price']).astype(int)
    df.dropna(subset=['next_day_close'], inplace=True)
    df['target'] = df['target'].astype(int)
    print("Target variable created.")
    return df

#### Calculate Historical Payouts
Estimate average upside “b” per winning trade for Kelly sizing. 

Only looks at wins; losses are implicit in Kelly’s (1-p)/b term.


In [13]:
def calculate_historical_payouts(df):
    """
    Returns a dict per ticker with:
      - 'b': avg_win / avg_loss_abs  (Kelly 'odds' ratio)
      - 'avg_win': mean positive next-day return when target==1
      - 'avg_loss_abs': mean absolute negative next-day return when target==0
    """
    out = {}
    for t, g in df.groupby('ticker'):
        # next_day return already implicit in your create_target_variable
        r = (g['next_day_close'] - g['close_price']) / g['close_price']
        wins = r[g['target'] == 1]
        losses = r[g['target'] == 0]
        avg_win = wins[wins > 0].mean() if (wins > 0).any() else np.nan
        avg_loss_abs = (-losses[losses < 0]).mean() if (losses < 0).any() else np.nan

        if np.isfinite(avg_win) and np.isfinite(avg_loss_abs) and avg_loss_abs > 0:
            b = avg_win / avg_loss_abs
        else:
            b = np.nan

        out[t] = {'b': b, 'avg_win': avg_win, 'avg_loss_abs': avg_loss_abs}
    return out


##### PyTorch Helpers

In [14]:
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
    def __len__(self): return self.X.shape[0]
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class BiLSTMClassifier(nn.Module):
    # PyTorch doesn't have recurrent_dropout; we use a small dropout between layers instead.
    def __init__(self, n_features, hidden1=128, hidden2=64, fc=32, dropout=0.3, inter_dropout=0.1):
        super().__init__()
        self.lstm1 = nn.LSTM(
            input_size=n_features, hidden_size=hidden1,
            num_layers=1, batch_first=True, bidirectional=True, dropout=0.0
        )
        self.inter_drop = nn.Dropout(p=inter_dropout)  # proxy for recurrent_dropout
        self.lstm2 = nn.LSTM(
            input_size=hidden1*2, hidden_size=hidden2,
            num_layers=1, batch_first=True, bidirectional=True, dropout=0.0
        )
        self.bn = nn.BatchNorm1d(num_features=hidden2*2)
        self.fc1 = nn.Linear(hidden2*2, fc)
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(p=dropout)
        self.fc_out = nn.Linear(fc, 1)  # logits

    def forward(self, x):
        out1, _ = self.lstm1(x)          # (B, T, 2*hidden1)
        out1 = self.inter_drop(out1)
        out2, _ = self.lstm2(out1)       # (B, T, 2*hidden2)
        last = out2[:, -1, :]            # (B, 2*hidden2)
        last = self.bn(last)
        z = self.fc1(last)
        z = self.relu(z)
        z = self.drop(z)
        logits = self.fc_out(z).squeeze(-1)  # (B,)
        return logits

class EarlyStopper:
    def __init__(self, patience=15, mode='min'):
        self.patience = patience
        self.counter = 0
        self.best_metric = None
        self.best_state_dict = None
        self.mode = mode  # 'min' for val_loss
    def step(self, metric, model):
        if self.best_metric is None:
            improved = True
        else:
            improved = (metric < self.best_metric) if self.mode == 'min' else (metric > self.best_metric)
        if improved:
            self.best_metric = metric
            self.best_state_dict = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            self.counter = 0
        else:
            self.counter += 1
        return improved

@torch.no_grad()
def _evaluate(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    all_logits = []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        total_loss += loss.item() * xb.size(0)
        all_logits.append(logits.detach().cpu())
    avg_loss = total_loss / len(loader.dataset)
    logits = torch.cat(all_logits, dim=0).numpy()
    probs = 1.0 / (1.0 + np.exp(-logits))  # sigmoid
    return avg_loss, probs

#### Train Company Models
Train, calibrate, and persist a per-ticker classifier.
This encapsulates per-asset modelling with proper scaling and probability calibration, which is crucial since Kelly needs probabilities (not raw logits).

In [15]:
def train_company_models(company_data_df: pd.DataFrame,
                         ticker: str,
                         feature_cols: list,
                         model_save_path: str,
                         sequence_length: int = None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # --- sequence length & guards (identical logic) ---
    if sequence_length is None:
        sequence_length = calculate_dynamic_sequence_length(company_data_df)

    if len(company_data_df) < sequence_length + 10:
        print(f"Not enough data for {ticker}.")
        return False, sequence_length

    contiguous_periods = identify_contiguous_periods(company_data_df)
    if not contiguous_periods:
        print(f"No contiguous periods found for {ticker}.")
        return False, sequence_length

    # --- build sequences (identical sources) ---
    targets = company_data_df['target'].values
    X, y = create_contiguous_sequences(
        company_data_df[feature_cols].values,
        targets,
        contiguous_periods,
        sequence_length
    )
    print(f"Created {len(X)} sequences for {ticker}.")
    if len(X) < 2:
        return False, sequence_length

    # --- chronological split (no shuffle) ---
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    # --- scale features (fit on train, transform both) ---
    scaler = MinMaxScaler()
    F = X_train.shape[-1]
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, F)).reshape(X_train.shape)
    X_val_scaled   = scaler.transform(  X_val.reshape(-1, F)).reshape(X_val.shape)

    # Keras fit used the scaled data (intended). We do the same:
    train_ds = SequenceDataset(X_train_scaled, y_train)
    val_ds   = SequenceDataset(X_val_scaled,   y_val)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=False,  drop_last=False)
    val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, drop_last=False)

    n_features = X_train.shape[-1]
    model = BiLSTMClassifier(n_features=n_features, dropout=0.3, inter_dropout=0.1).to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # clipnorm via grad clipping below
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=7, min_lr=1e-7)
    early = EarlyStopper(patience=15, mode='min')

    # --- training loop: up to 100 epochs, grad-clip=1.0, silent like verbose=0 ---
    for epoch in range(100):
        model.train()
        total_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # clipnorm=1.0
            optimizer.step()
            total_loss += loss.item() * xb.size(0)

        train_loss = total_loss / len(train_loader.dataset)
        val_loss, _ = _evaluate(model, val_loader, device, loss_fn)

        scheduler.step(val_loss)  # ReduceLROnPlateau on val_loss

        # EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
        _ = early.step(val_loss, model)
        if early.counter >= early.patience:  # stop after 'patience' non-improve epochs
            break

    # Restore best weights
    if early.best_state_dict is not None:
        model.load_state_dict(early.best_state_dict)

    # --- final validation pass for isotonic calibration (on scaled val data) ---
    _, validation_predictions = _evaluate(model, val_loader, device, loss_fn)

    # Keras code used 10% buffer (comment said 5% but code was 0.1). We match the code:
    pred_min, pred_max = validation_predictions.min(), validation_predictions.max()
    buffer = 0.1 * (pred_max - pred_min)
    y_min_dynamic = max(0.0, pred_min - buffer)
    y_max_dynamic = min(1.0, pred_max + buffer)

    calibrator = IsotonicRegression(y_min=y_min_dynamic, y_max=y_max_dynamic, out_of_bounds='clip')
    calibrator.fit(validation_predictions, y_val.astype(float))

    os.makedirs(model_save_path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(model_save_path, f"{ticker}_lstm.pt"))
    joblib.dump(calibrator,        os.path.join(model_save_path, f"{ticker}_calibrator.pkl"))
    joblib.dump(scaler,            os.path.join(model_save_path, f"{ticker}_scaler.pkl"))
    joblib.dump(sequence_length,   os.path.join(model_save_path, f"{ticker}_seq_length.pkl"))

    del model, scaler, calibrator, train_loader, val_loader, train_ds, val_ds
    del X_train, X_val, y_train, y_val, X_train_scaled, X_val_scaled
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return True, sequence_length

#### Predict Next Day Performance 
One-step-ahead inference for a ticker on a given day.

Feeds calibrated p(up) into the portfolio/Kelly step.

In [16]:
def predict_next_day_performance(company_data_df: pd.DataFrame,
                                 ticker: str,
                                 feature_cols: list,
                                 model_save_path: str) -> dict | None:

    try:
        # Load artifacts
        state_dict_path = os.path.join(model_save_path, f"{ticker}_lstm.pt")
        calibrator = joblib.load(os.path.join(model_save_path, f"{ticker}_calibrator.pkl"))
        scaler = joblib.load(os.path.join(model_save_path, f"{ticker}_scaler.pkl"))
        sequence_length = joblib.load(os.path.join(model_save_path, f"{ticker}_seq_length.pkl"))

        if not os.path.isfile(state_dict_path):
            return None
        
        if hasattr(scaler, "n_features_in_") and scaler.n_features_in_ != len(feature_cols):
            return None

        # Build model skeleton with correct input dim, then load weights
        n_features = len(feature_cols)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = BiLSTMClassifier(n_features=n_features).to(device)
        model.load_state_dict(torch.load(state_dict_path, map_location=device))
        model.eval()
    except Exception:
        return None

    # Find the last contiguous block and take the most recent sequence_length rows
    contiguous_periods = identify_contiguous_periods(company_data_df)
    if not contiguous_periods:
        return None

    last_start, last_end = contiguous_periods[-1]
    period_data = company_data_df.iloc[last_start:last_end + 1]
    if len(period_data) < sequence_length:
        return None

    last_sequence = period_data.tail(sequence_length)

    # Scale using the saved scaler (fit  train only)
    # Ensure column order matches training
    try:
        scaled_features = scaler.transform(last_sequence[feature_cols])
    except Exception:
        return None

    # Shape: (1, T, F) as float32 tensor
    x = torch.from_numpy(np.asarray([scaled_features], dtype=np.float32)).to(device)

    with torch.no_grad():
        logits = model(x)                  # shape (1,)
        prob = torch.sigmoid(logits).item()  # scalar in [0,1]

    # Isotonic calibration (expects iterable)
    calibrated = float(calibrator.predict([prob])[0])

    print(f"Raw prediction for {ticker}: {prob}")
    print(f"Calibrated prediction for {ticker}: {calibrated}")

    return {
        'ticker': ticker,
        'raw_prediction': prob,
        'calibrated_prediction': calibrated
    }

#### Select and Size Portfolio
Turn a cross-section of calibrated predictions into position sizes.

Enforces diversification (one per sector) and risk discipline (Fractional Kelly).

In [17]:
def select_and_size_portfolio(daily_predictions_df: pd.DataFrame, payout_map: dict,
                            total_capital: float, sector_threshold: float,
                            kelly_fraction: float) -> pd.DataFrame:

    investment_decisions = []

    if daily_predictions_df.empty:
        return pd.DataFrame()

    for sector, group in daily_predictions_df.groupby('sector'):
        avg_sector_score = group['calibrated_prediction'].mean()
        if avg_sector_score < sector_threshold:
            continue

        best_stock_in_sector = group.loc[group['calibrated_prediction'].idxmax()]
        ticker = best_stock_in_sector['ticker']
        p = best_stock_in_sector['calibrated_prediction']
        b_info = payout_map.get(ticker)
        
        if not b_info or not np.isfinite(b_info.get('b', np.nan)): 
            continue
        b = max(b_info['b'], 1e-6)  # avoid div-by-zero
        kelly_percentage = p - (1 - p) / b

        if kelly_percentage > 0:
            investment_fraction = kelly_percentage * kelly_fraction
            investment_amount = total_capital * investment_fraction

            print(f"Investing {investment_amount} in {ticker}")

            investment_decisions.append({
                'ticker': ticker,
                'investment_fraction': investment_fraction,
                'investment_amount': investment_amount,
                'predicted_prob': p
            })

    return pd.DataFrame(investment_decisions)

#### Models Exist for Ticker
Quick guard to avoid retraining if a ticker’s artifacts already exist.

Decide whether to train.

In [18]:
def models_exist_for_ticker(ticker, model_path):
  
    model_file = os.path.join(model_path, f"{ticker}_lstm.pt")
    calibrator_file = os.path.join(model_path, f"{ticker}_calibrator.pkl")
    scaler_file = os.path.join(model_path, f"{ticker}_scaler.pkl")
    seq_length_file = os.path.join(model_path, f"{ticker}_seq_length.pkl")

    return (os.path.exists(model_file) and
            os.path.exists(calibrator_file) and
            os.path.exists(scaler_file) and
            os.path.exists(seq_length_file))

#### Run Simulation
The “engine” — walks forward over dates, trains as needed, predicts, sizes, books PnL, and updates capital.

Enforces chronological integrity, periodic retraining, diversification, and proper capital tracking.

In [19]:
def run_simulation(master_df: pd.DataFrame, payout_map: dict, feature_cols: list,
                  initial_capital: float, initial_training_days: int = INITIAL_TRAINING_DAYS):

    print(f"\nStarting simulation with {initial_training_days} initial training days...")
    capital = initial_capital
    simulation_log = []

    all_tickers = master_df['ticker'].unique()
    retrain_counter = {ticker: RETRAIN_INTERVAL for ticker in all_tickers}
    ticker_sequence_lengths = {}

    unique_dates = sorted(master_df['date'].unique())


    start_index = min(initial_training_days, len(unique_dates) - 1)

    print(f"Starting predictions from day {start_index} (after {initial_training_days} training days)")

    for i in tqdm(range(start_index, len(unique_dates)), desc="Simulating Trading Days"):
        current_date = unique_dates[i]

        historical_data = master_df[master_df['date'] < current_date]
        todays_data_for_prediction = master_df[master_df['date'] == unique_dates[i-1]]
        next_day_data = master_df[master_df['date'] == current_date]

        daily_predictions = []

        for ticker in todays_data_for_prediction['ticker'].unique():
            company_hist_data = historical_data[historical_data['ticker'] == ticker]
            if company_hist_data.empty:
                continue


            if retrain_counter.get(ticker, 0) >= RETRAIN_INTERVAL or not models_exist_for_ticker(ticker, MODEL_SAVE_PATH):
                training_success, seq_length = train_company_models(
                    company_hist_data, ticker, feature_cols, MODEL_SAVE_PATH
                )
                if training_success:
                    ticker_sequence_lengths[ticker] = seq_length
                    retrain_counter[ticker] = 0
                else:
                    continue

            prediction_result = predict_next_day_performance(
                company_hist_data, ticker, feature_cols, MODEL_SAVE_PATH
            )

            if prediction_result:
                info = todays_data_for_prediction[todays_data_for_prediction['ticker'] == ticker].iloc[0]
                prediction_result.update({'company_name': info['company_name'], 'sector': info['sector']})
                daily_predictions.append(prediction_result)
                retrain_counter[ticker] += 1

        daily_predictions_df = pd.DataFrame(daily_predictions)
        investment_decision_df = select_and_size_portfolio(
            daily_predictions_df, payout_map, capital, SECTOR_CONFIDENCE_THRESHOLD, KELLY_FRACTION
        )

        capital_at_start_of_day = capital
        total_pnl = 0
        invested_capital = 0

        if not investment_decision_df.empty:
            invested_capital = investment_decision_df['investment_amount'].sum()
            capital -= invested_capital

            for _, trade in investment_decision_df.iterrows():
                ticker = trade['ticker']
                investment_amount = trade['investment_amount']
                outcome = next_day_data[next_day_data['ticker'] == ticker]
                if not outcome.empty:
                    prev_close = todays_data_for_prediction[todays_data_for_prediction['ticker'] == ticker].iloc[0]['close_price']
                    actual_return = outcome.iloc[0]['close_price'] / prev_close - 1
                    pnl = investment_amount * actual_return
                    total_pnl += pnl
                    capital += (investment_amount + pnl)

        simulation_log.append({
            'date': current_date,
            'capital_start': capital_at_start_of_day,
            'capital_end': capital,
            'daily_pnl': total_pnl,
            'investments_made': investment_decision_df.to_dict('records')
        })

    return pd.DataFrame(simulation_log)

#### Calculate Final Results


In [20]:
def calculate_final_results(simulation_log: pd.DataFrame, initial_capital: float):

    if simulation_log.empty:
        print("Simulation log is empty. No results to calculate.")
        return

    final_capital = simulation_log['capital_end'].iloc[-1]
    total_roi = (final_capital - initial_capital) / initial_capital
    simulation_log['daily_return'] = (simulation_log['capital_end'] / simulation_log['capital_start']) - 1

    if simulation_log['daily_return'].std() > 0:
        sharpe_ratio = (simulation_log['daily_return'].mean() / simulation_log['daily_return'].std()) * np.sqrt(252)
    else:
        sharpe_ratio = 0.0

    print("\n--- Simulation Results ---")
    print(f"Initial Capital: ${initial_capital:,.2f}")
    print(f"Final Capital:   ${final_capital:,.2f}")
    print(f"Total Return on Investment (ROI): {total_roi:.2%}")
    print(f"Annualized Sharpe Ratio: {sharpe_ratio:.2f}")
    print("--------------------------")

In [21]:
companies = pd.read_parquet('stocknet-dataset/stock_table.parquet')
tweets = pd.read_parquet('stocknet-dataset/stock_tweets_withsentiment_withemotion_withstance_nomerge.parquet')
stocks = pd.read_parquet('stocknet-dataset/stock_prices.parquet')

companies = companies.rename(columns={'symbol': 'ticker'})

companies.columns = [x.lower() for x in companies.columns]
tweets.columns = [x.lower() for x in tweets.columns]
stocks.columns = [x.lower() for x in stocks.columns]

tweets['stance_positive'] = (tweets['stance_label'] == 'Positive').astype(int)
tweets['stance_negative'] = (tweets['stance_label'] == 'Negative').astype(int)

tweets_merged = tweets.groupby(['date', 'ticker'], as_index=False).agg({
    'text': lambda x: ' '.join(x),
    'sentiment': lambda x: x.mean(),
    'emotion_anger': 'sum',
    'emotion_disgust': 'sum',
    'emotion_fear': 'sum',
    'emotion_joy': 'sum',
    'emotion_neutral': 'sum',
    'emotion_sadness': 'sum',
    'emotion_surprize': 'sum',
    'stance_positive': 'sum',
    'stance_negative': 'sum'
})




tweets_merged['date'] = pd.to_datetime(tweets_merged['date'])
stocks['date'] = pd.to_datetime(stocks['date'])

"""
master_df = stocks.merge(
    tweets_merged,
    on=["date", "ticker"]
)
"""


master_df = pd.merge(
    stocks,
    tweets_merged,
    on=["date", "ticker"],
    how='left'
)

tweet_feature_cols = ['sentiment', 'emotion_anger', 'emotion_disgust', 'emotion_fear', 'emotion_joy', 'emotion_neutral', 'emotion_sadness', 'emotion_surprize', 'stance_positive', 'stance_negative']
for col in tweet_feature_cols:
    if col in master_df.columns:
        master_df[col].fillna(0, inplace=True)



companies = companies.rename(columns={'symbol': 'ticker'})

master_df = pd.merge(master_df, companies[['ticker', 'sector', 'company']], on='ticker', how='left')


feature_cols = ['open','high','low','volume']

master_df = master_df.rename(columns={'close': 'close_price', 'company': 'company_name'})


print(f"Shape of master_df before dropping NaNs: {master_df.shape}")
#master_df.dropna(inplace=True)
print(f"Shape of master_df after dropping NaNs: {master_df.shape}")

master_df.rename(columns={'close_price': 'close'}, inplace=True)





master_df.sort_values(by=['ticker', 'date'], inplace=True)


def apply_ta_indicators(df_group):
    df_group.set_index(pd.DatetimeIndex(df_group['date']), inplace=True)
    #Trend
    df_group.ta.ema(length=12, append=True)
    df_group.ta.ema(length=26, append=True)
    df_group.ta.ema(length=50, append=True)

    df_group.ta.macd(fast=12, slow=26, signal=9, append=True)



    df_group.ta.rsi(length=14, append=True)
    df_group.ta.stochrsi(length=14, append=True)


    df_group.ta.atr(length=14, append=True)

    bb = ta.bbands(df_group['close'], length=20, std=2)
    df_group['BB_upper'] = bb['BBU_20_2.0']
    df_group['BB_middle'] = bb['BBM_20_2.0']
    df_group['BB_lower'] = bb['BBL_20_2.0']


    df_group.ta.obv(append=True)
    return df_group.reset_index(drop=True)

master_df = master_df.groupby('ticker').apply(apply_ta_indicators)


Shape of master_df before dropping NaNs: (108592, 21)
Shape of master_df after dropping NaNs: (108592, 21)


In [22]:
master_df.drop(columns=['text','adj close','sentiment','emotion_anger','emotion_disgust','emotion_fear','emotion_joy','emotion_neutral','emotion_sadness','emotion_surprize'], inplace=True)


In [23]:
master_df


date       open       high        low      close  \
ticker                                                               
AAPL   0    2012-09-04  95.108574  96.448570  94.928574  96.424286   
       1    2012-09-05  96.510002  96.621429  95.657143  95.747147   
       2    2012-09-06  96.167145  96.898575  95.828575  96.610001   
       3    2012-09-07  96.864288  97.497147  96.538574  97.205711   
       4    2012-09-10  97.207146  97.612854  94.585716  94.677139   
...                ...        ...        ...        ...        ...   
XOM    1253 2017-08-28  76.900002  76.940002  76.260002  76.470001   
       1254 2017-08-29  76.209999  76.489998  76.080002  76.449997   
       1255 2017-08-30  76.239998  76.449997  76.059998  76.099998   
       1256 2017-08-31  76.269997  76.489998  76.050003  76.330002   
       1257 2017-09-01  76.370003  76.849998  76.320000  76.570000   

                  volume ticker  stance_positive  stance_negative  \
ticker                                                              
AAPL   0      91973000.0   AAPL              0.0              0.0   
       1      84093800.0   AAPL              0.0              0.0   
       2      97799100.0   AAPL              0.0              0.0   
       3      82416600.0   AAPL              0.0              0.0   
       4     121999500.0   AAPL              0.0              0.0   
...                  ...    ...              ...              ...   
XOM    1253    8229700.0    XOM              0.0              0.0   
       1254    7060400.0    XOM              0.0              0.0   
       1255    8218000.0    XOM              0.0              0.0   
       1256   15641700.0    XOM              0.0              0.0   
       1257    7340800.0    XOM              0.0              0.0   

                       sector  ... MACDh_12_26_9  MACDs_12_26_9     RSI_14  \
ticker                         ...                                           
AAPL   0       Consumer Goods  ...           NaN            NaN        NaN   
       1       Consumer Goods  ...           NaN            NaN        NaN   
       2       Consumer Goods  ...           NaN            NaN        NaN   
       3       Consumer Goods  ...           NaN            NaN        NaN   
       4       Consumer Goods  ...           NaN            NaN        NaN   
...                       ...  ...           ...            ...        ...   
XOM    1253  Basic Matierials  ...     -0.107548      -0.972858  31.975492   
       1254  Basic Matierials  ...     -0.069077      -0.990127  31.851847   
       1255  Basic Matierials  ...     -0.054652      -1.003790  29.688704   
       1256  Basic Matierials  ...     -0.018917      -1.008519  32.913052   
       1257  Basic Matierials  ...      0.028984      -1.001273  36.200731   

             STOCHRSIk_14_14_3_3  STOCHRSId_14_14_3_3   ATRr_14   BB_upper  \
ticker                                                                       
AAPL   0                     NaN                  NaN       NaN        NaN   
       1                     NaN                  NaN       NaN        NaN   
       2                     NaN                  NaN       NaN        NaN   
       3                     NaN                  NaN       NaN        NaN   
       4                     NaN                  NaN       NaN        NaN   
...                          ...                  ...       ...        ...   
XOM    1253            35.117121            31.775404  0.786087  81.525829   
       1254            48.597552            38.712818  0.759224  81.303475   
       1255            55.025431            46.246701  0.732850  80.964170   
       1256            73.940933            59.187972  0.711932  80.569554   
       1257            85.986580            71.650981  0.698937  80.167620   

             BB_middle   BB_lower          OBV  
ticker                                          
AAPL   0           NaN        NaN   91973000.0  
       1           NaN        Na

In [24]:
columns_to_check = ['EMA_12', 'EMA_26','EMA_50','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','RSI_14','ATRr_14','STOCHRSIk_14_14_3_3','STOCHRSId_14_14_3_3','ATRr_14','BB_upper','BB_middle','BB_lower','OBV']
master_df = master_df.dropna(subset=columns_to_check)

In [25]:
feature_cols = ['open','high','low','volume',
                'stance_positive','stance_negative'
                ]

new_indicator_columns = [
    'EMA_12', 'EMA_26', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
    'RSI_14', 'ATRr_14', 'STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3',
    'BB_upper', 'BB_middle', 'BB_lower', 'OBV'
]
feature_cols.extend(new_indicator_columns)
print(f"Final feature columns: {feature_cols}")

Final feature columns: ['open', 'high', 'low', 'volume', 'stance_positive', 'stance_negative', 'EMA_12', 'EMA_26', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_14', 'ATRr_14', 'STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3', 'BB_upper', 'BB_middle', 'BB_lower', 'OBV']


In [26]:
master_df = master_df.rename(columns={'close': 'close_price', 'company': 'company_name'})


In [27]:
master_df.reset_index(drop=True, inplace=True)
master_df

,date,open,high,low,close_price,volume,ticker,stance_positive,stance_negative,sector,...,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,ATRr_14,BB_upper,BB_middle,BB_lower,OBV
0,2012-11-14,77.928574,78.207146,76.597145,76.697144,119292600.0,AAPL,0.0,0.0,Consumer Goods,...,-0.538077,-3.703588,25.771012,21.054629,19.582354,2.377852,94.648550,84.401357,74.154164,-1.014356e+09
1,2012-11-15,76.790001,77.071426,74.660004,75.088570,197477700.0,AAPL,0.0,0.0,Consumer Goods,...,-0.537725,-3.838019,23.573491,15.792949,19.993462,2.380310,93.761634,83.514428,73.267223,-1.211834e+09
2,2012-11-16,75.028572,75.714287,72.250000,75.382858,316723400.0,AAPL,0.0,0.0,Consumer Goods,...,-0.455544,-3.951905,24.836267,12.243346,16.363641,2.459547,92.716200,82.679214,72.642228,-8.951103e+08
3,2012-11-19,77.244286,81.071426,77.125717,80.818573,205829400.0,AAPL,0.0,0.0,Consumer Goods,...,0.002769,-3.951213,43.429047,39.692073,22.576123,2.695187,91.617665,82.201285,72.784906,-6.892809e+08
4,2012-11-20,81.701431,81.707146,79.225716,80.129997,160688500.0,AAPL,0.0,0.0,Consumer Goods,...,0.281964,-3.880722,42.011353,69.373201,40.436207,2.679612,91.027780,81.851785,72.675790,-8.499694e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104215,2017-08-28,76.900002,76.940002,76.260002,76.470001,8229700.0,XOM,0.0,0.0,Basic Matierials,...,-0.107548,-0.972858,31.975492,35.117121,31.775404,0.786087,81.525829,78.243500,74.961171,-2.688251e+08
104216,2017-08-29,76.209999,76.489998,76.080002,76.449997,7060400.0,XOM,0.0,0.0,Basic Matierials,...,-0.069077,-0.990127,31.851847,48.597552,38.712818,0.759224,81.303475,78.057500,74.811525,-2.758855e+08
104217,2017-08-30,76.239998,76.449997,76.059998,76.099998,8218000.0,XOM,0.0,0.0,Basic Matierials,...,-0.054652,-1.003790,29.688704,55.025431,46.246701,0.732850,80.964170,77.832500,74.700830,-2.841035e+08
104218,2017-08-31,76.269997,76.489998,76.050003,76.330002,15641700.0,XOM,0.0,0.0,Basic Matierials,...,-0.018917,-1.008519,32.913052,73.940933,59.187972,0.711932,80.569554,77.624500,74.679446,-2.684618e+08


In [28]:

master_df_with_target = create_target_variable(master_df)
payout_map = calculate_historical_payouts(master_df_with_target)

initial_capital = 100_000.0
simulation_results = run_simulation(
    master_df_with_target,
    payout_map,
    feature_cols,
    initial_capital,
    initial_training_days=INITIAL_TRAINING_DAYS
)

display(simulation_results.head(100))
calculate_final_results(simulation_results, initial_capital)

Creating target variable...
Target variable created.

Starting simulation with 1100 initial training days...
Starting predictions from day 1100 (after 1100 training days)


Simulating Trading Days:   0%|          | 0/108 [00:00<?, ?it/s]

Created 1088 sequences for AAPL.
Raw prediction for AAPL: 0.47039496898651123
Calibrated prediction for AAPL: 0.5018218159675598
Created 1088 sequences for ABB.
Raw prediction for ABB: 0.4240192174911499
Calibrated prediction for ABB: 0.4763040542602539
Created 1007 sequences for ABBV.
Raw prediction for ABBV: 0.42459937930107117
Calibrated prediction for ABBV: 0.39526522159576416
Created 1088 sequences for AEP.
Raw prediction for AEP: 0.4913913309574127
Calibrated prediction for AEP: 0.5
Created 531 sequences for AGFS.
Raw prediction for AGFS: 0.5259708166122437
Calibrated prediction for AGFS: 0.5769230723381042
Created 1088 sequences for AMGN.
Raw prediction for AMGN: 0.5276541709899902
Calibrated prediction for AMGN: 0.5454545617103577
Created 1088 sequences for AMZN.
Raw prediction for AMZN: 0.4969283640384674
Calibrated prediction for AMZN: 0.5879396796226501
Created 1088 sequences for BA.
Raw prediction for BA: 0.5016956329345703
Calibrated prediction for BA: 0.5196605920791626
C

Simulating Trading Days:   1%|          | 1/108 [11:52<21:10:11, 712.26s/it]

Raw prediction for XOM: 0.46825194358825684
Calibrated prediction for XOM: 0.517439067363739
Investing 588.8473752494231 in BBL
Investing 971.2207014870437 in AGFS
Investing 788.63606124389 in TM
Investing 807.7199216411868 in MA
Investing 515.1414553795569 in AMGN
Investing 508.27282578488956 in GD
Investing 1037.6658618305605 in AMZN
Investing 386.8775412083564 in TSM
Investing 3.940418464623008 in EXC
Raw prediction for AAPL: 0.47171342372894287
Calibrated prediction for AAPL: 0.5018218159675598
Raw prediction for ABB: 0.4236283004283905
Calibrated prediction for ABB: 0.4763040542602539
Raw prediction for ABBV: 0.4275580942630768
Calibrated prediction for ABBV: 0.39526522159576416
Raw prediction for AEP: 0.4914591312408447
Calibrated prediction for AEP: 0.504379153251648
Raw prediction for AGFS: 0.5145886540412903
Calibrated prediction for AGFS: 0.5769230723381042
Raw prediction for AMGN: 0.5289837718009949
Calibrated prediction for AMGN: 0.5502575039863586
Raw prediction for AMZN: 

...

Raw prediction for XOM: 0.486232191324234
Calibrated prediction for XOM: 0.5202333331108093
Investing 1332.2794005053854 in SLB
Investing 213.56024038131855 in CODI
Investing 388.5922826237039 in MO
Investing 599.7765908747798 in JPM
Investing 278.6281343317361 in JNJ
Investing 511.944505184222 in GD
Investing 1045.1617895587256 in AMZN
Investing 1443.8467810177397 in GOOG
Investing 25.673238549230895 in SO
Raw prediction for AAPL: 0.47021913528442383
Calibrated prediction for AAPL: 0.5018218159675598
Raw prediction for ABB: 0.438707560300827
Calibrated prediction for ABB: 0.4763040542602539
Raw prediction for ABBV: 0.36118200421333313
Calibrated prediction for ABBV: 0.39526522159576416
Raw prediction for AEP: 0.4847904145717621
Calibrated prediction for AEP: 0.4851284921169281
Raw prediction for AGFS: 0.2981356978416443
Calibrated prediction for AGFS: 0.39688241481781006
Raw prediction for AMGN: 0.5227975249290466
Calibrated prediction for AMGN: 0.5111111402511597
Raw prediction for A

Simulating Trading Days: 100%|██████████| 108/108 [14:47<00:00,  8.22s/it]

Raw prediction for TSM: 0.22423361241817474
Calibrated prediction for TSM: 0.5259740352630615
Raw prediction for UL: 0.4422267973423004
Calibrated prediction for UL: 0.45924845337867737
Raw prediction for UN: 0.465093731880188
Calibrated prediction for UN: 0.4751264154911041
Raw prediction for UNH: 0.4500127136707306
Calibrated prediction for UNH: 0.45060208439826965
Raw prediction for UPS: 0.479065865278244
Calibrated prediction for UPS: 0.4749999940395355
Raw prediction for UTX: 0.2576141059398651
Calibrated prediction for UTX: 0.18146172165870667
Raw prediction for V: 0.4964347779750824
Calibrated prediction for V: 0.5080437660217285
Raw prediction for VZ: 0.46763724088668823
Calibrated prediction for VZ: 0.4730283319950104
Raw prediction for WFC: 0.4749022126197815
Calibrated prediction for WFC: 0.49569568037986755
Raw prediction for WMT: 0.38617223501205444
Calibrated prediction for WMT: 0.4571428596973419
Raw prediction for XOM: 0.4834703505039215
Calibrated prediction for XOM: 0

,date,capital_start,capital_end,daily_pnl,investments_made
0,2017-03-30,100000.000000,100037.216440,37.216440,"[{'ticker': 'BBL', 'investment_fraction': 0.00..."
1,2017-03-31,100037.216440,100050.816506,13.600067,"[{'ticker': 'BP', 'investment_fraction': 0.005..."
2,2017-04-03,100050.816506,100049.378592,-1.437914,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."
3,2017-04-04,100049.378592,100082.315351,32.936759,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."
4,2017-04-05,100082.315351,100055.723006,-26.592345,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."
...,...,...,...,...,...
95,2017-08-15,100767.921142,100767.245628,-0.675514,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."
96,2017-08-16,100767.245628,100783.960233,16.714605,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."
97,2017-08-17,100783.960233,100689.340042,-94.620191,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."
98,2017-08-18,100689.340042,100690.237917,0.897876,"[{'ticker': 'SLB', 'investment_fraction': 0.01..."



--- Simulation Results ---
Initial Capital: $100,000.00
Final Capital:   $100,775.94
Total Return on Investment (ROI): 0.78%
Annualized Sharpe Ratio: 2.74
--------------------------


In [29]:
display(simulation_results.head(100))

,date,capital_start,capital_end,daily_pnl,investments_made,daily_return
0,2017-03-30,100000.000000,100037.216440,37.216440,"[{'ticker': 'BBL', 'investment_fraction': 0.00...",0.000372
1,2017-03-31,100037.216440,100050.816506,13.600067,"[{'ticker': 'BP', 'investment_fraction': 0.005...",0.000136
2,2017-04-03,100050.816506,100049.378592,-1.437914,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",-0.000014
3,2017-04-04,100049.378592,100082.315351,32.936759,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",0.000329
4,2017-04-05,100082.315351,100055.723006,-26.592345,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",-0.000266
...,...,...,...,...,...,...
95,2017-08-15,100767.921142,100767.245628,-0.675514,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",-0.000007
96,2017-08-16,100767.245628,100783.960233,16.714605,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",0.000166
97,2017-08-17,100783.960233,100689.340042,-94.620191,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",-0.000939
98,2017-08-18,100689.340042,100690.237917,0.897876,"[{'ticker': 'SLB', 'investment_fraction': 0.01...",0.000009


In [30]:
folder_path = 'trained_models/'
if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
else:
    print(f"Folder not found: {folder_path}")

print(f"Contents of {folder_path} after deletion attempt:")
if os.path.exists(folder_path):
    print(os.listdir(folder_path))
else:
    print("Folder does not exist.")

Contents of trained_models/ after deletion attempt:
[]
